# Topic Modelling

## Libraries

In [1]:
import pandas as pd
import numpy as np
import nltk
import re
import gensim
import pyLDAvis.gensim
import warnings

from gensim import corpora, models
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

warnings.filterwarnings('ignore', category=DeprecationWarning, message='.*use @default decorator instead.*')

/Users/tomewing/anaconda/lib/python3.5/site-packages/funcy/decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
/Users/tomewing/anaconda/lib/python3.5/site-packages/funcy/decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
/Users/tomewing/anaconda/lib/python3.5/site-packages/funcy/decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
/Users/tomewing/anaconda/lib/python3.5/site-packages/funcy/decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
/Users/tomewing/anaconda/lib/python3.5/site-packages/funcy/decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
/User

## Options

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))       # Changing the cell widths

pd.options.display.max_rows = 30                                            # Setting the max number of rows
pd.options.display.max_columns = 50                                         # Setting the max number of columns

pyLDAvis.enable_notebook()

## Variables

In [17]:
path = '/Users/tomewing/Datadive/Data/'    # Data Directory 
out = '/Users/tomewing/Datadive/Datadive/Outputs/'  # Output Directory
msg = 'messages.csv'                       # Input Dataset

sample_size = 5000
topics = 10

## Import

In [18]:
df_msg_in = pd.read_csv(path + msg)
df_msg_en = df_msg_in[(df_msg_in['language'] == 'EN')] 

## Functions

In [19]:
def cleaner(row):
    '''Function to clean the text data and prep for further analysis'''
    text = row['body'].lower()                   # Converts to lower case
    text = re.sub("[^a-zA-Z]"," ",text)          # Removes punctuation
    text = re.sub("cyclist","cycl",text)         # Manual intervention for 'cyclist'
    text = text.split()                          # Splits the data into individual words 
    text = [w for w in text if not w in stops]   # Removes stopwords
    text = [p_stemmer.stem(i) for i in text]     # Stemming (reducing words to their root)
    return text

def model(data):
    stops = set(stopwords.words("english"))     # Creating a set of Stopwords
    p_stemmer = PorterStemmer()                 # Creating the stemmer model
    data_dict = corpora.Dictionary(data)                       # Creates an id <-> term dictionary
    data_corpus = [data_dict.doc2bow(text) for text in data]     # convert tokenized documents into a document-term matrix
    data_model = gensim.models.ldamodel.LdaModel(data_corpus, 
                                                   num_topics=topics, 
                                                   id2word = data_dict,
                                                   passes=20)        #  generate LDA model

    data_vis = pyLDAvis.gensim.prepare(data_model, data_corpus, data_dict)        # Visualise LDA Model
    pyLDAvis.save_html(data=data_vis,
                       fileobj=out + 'Data_vis.html')
    data_vis

In [ ]:
df_msg_en['Clean'] = df_msg_en.apply(cleaner,axis=1)

df_msg_en_q = df_msg_en[(df_msg_en['type'] == 'Q')]
df_msg_en_a = df_msg_en[(df_msg_en['type'] == 'A')]

li_msg_en_q = df_msg_en_q['Clean'].tolist()
li_msg_en_a = df_msg_en_a['Clean'].tolist()

In [16]:
sample = li_msg_en_q[:sample_size]
model(sample)